In [ ]:
# default_exp core.baseline

## Baseline Analysis

> We are using the adjudidicated cases from 2018 to differentiate the cases by their Origin.  Our first step is to determine if the they have different status counts.  The conclusion of this analysis will driver futher analysis of years where there were significant policy changes made due to COVID.


In particular ...

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

file_path = r'C:\Users\mbeaulieu\rsi_project_book\Data\\'



In [ ]:
#export
import pandas as pd
import altair as alt


In [ ]:
#export
ORIGIN_MAPPINGS = {
        'DMER - DRIVER MEDICAL EXAMINATION REPORT': 'DMER', 
        'OTHER':'Other', 
        'PHYSICIAN':'Physician',
        'PUBLIC/FAMILY': 'Public/Family', 
        'EVF - EXAM OF VISUAL FUNCTIONS': 'EVF',
        'ICBC TRIGGER':'ICBC Trigger',
        'ICBC':'ICBC',
        'OTHER HEALTH PROFESSIONAL':'Other Health', 
        'POLICE':'Police', 
        'REHAB':'Rehab',
        'DRIVER IMPROVEMENT PROGRAM':'Driver Improvement',
        'DRIVER APPEAL REQUEST':'Appeal',
        'SEAT BELT EXEMPTION APPLICATION':'Seat Belt',
        'SPECIALIST REPORT':'Specialist Report',
        'MLA ENQUIRY':'MLA Enquiry'    
}


In [ ]:
#export

def get_fte_days_2018():
# We created three lists, containing names, ages, and locations, holding our ordered data
# We then created a Python zip() object, which contained the zips of names, ages, and locations. We then applied the list() function to turn this zip object into a list of tuples
# We then passed this zipped object into our DataFrame() class, along with a list of our column names to create our dataframe.

    months = ['Jan-2018', 'Feb-2018', 'Mar-2018', 'Apr-2018','May-2018', 'Jun-2018', 'Jul-2018', 'Aug-2018','Sep-2018', 'Oct-2018', 'Nov-2018', 'Dec-2018']
    ftes = [376
    ,311
    ,361
    ,332
    ,394
    ,360
    ,335
    ,366
    ,323
    ,380
    ,335
    ,281]


    zipped = list(zip(months, ftes))
    ftedays_df = pd.DataFrame(zipped, columns=['Month-Year', 'FTE (Days)'])
    ftedays_df

    d = ftedays_df.copy()
    d.columns = ['Month', 'FTE (Days)']
    
    return d


In [ ]:
#export
def derive_statuscount_percase(df_in, ftedays_df):
    FTE_DAYS_YEAR = ftedays_df['FTE (Days)'].sum()
    print('FTE_DAYS_YEAR', FTE_DAYS_YEAR)
    aggregation = {
        'Case Count': ('STATUS_COUNT','size'),
        'Total Status Change Count': ('STATUS_COUNT','sum')
    }    

    # df = df_in.groupby(['ORIGIN_DSC']) \
    # .agg(** aggregation).reset_index()

    # aggregation = {
    #     'Case Count': ('DRIVERS_LICENSE_NO','size'),
    #     'Status Count': ('STATUS_COUNT','sum'),
    #     }

    df = df_in.groupby([pd.Grouper(freq='M', key='CASE_OPENED_DT') , 
                                pd.Grouper(key='ORIGIN_DSC'),                                   
                                ]).agg(** aggregation)

    df = pd.DataFrame(df).reset_index()

    df['Opened Month'] = df.apply(lambda x: x['CASE_OPENED_DT'].strftime('%b') + '-' + x['CASE_OPENED_DT'].strftime('%Y'), axis=1)
    
    #case_summary_data['Total Cases in Group'] = case_summary_data.groupby(['Origin Report'])['Cases'].transform(lambda x: sum(x) )
    df['Total Cases In Month'] = df.groupby(['CASE_OPENED_DT'])['Case Count'].transform( lambda x: sum(x))
    df['Total Status Changes In Month'] = df.groupby(['CASE_OPENED_DT'])['Total Status Change Count'].transform( lambda x: sum(x))
    df['Monthly Status Changes/Case'] = df.apply(lambda x: x['Total Status Changes In Month']/x['Total Cases In Month']  , axis=1)


    df['Group Status Changes/Case'] = df.apply(lambda x: x['Total Status Change Count']/x['Case Count']  , axis=1)
    #Month-Year	FTE (Days)
    df = pd.merge(df, ftedays_df, how='left',  left_on='Opened Month', right_on='Month')
    #df = df[df['Is Adjudicated'] == 'Adjudicated']
    print('df shape ', df.shape)
    case_count = df['Case Count'].sum()
    status_change_count = df['Total Status Change Count'].sum()

    df['Group Case Count/FTE'] = df.apply(lambda x: x['Case Count']/x['FTE (Days)']  , axis=1)
    df['Group Status Change/FTE'] = df.apply(lambda x: x['Total Status Change Count']/x['FTE (Days)']  , axis=1)


    print(f"Adjudicated Case Count for 2018: {case_count:,}")
    print(f"Adjudicated Status Change Count for 2018: {status_change_count:,}" )
    print(f"Average Status Change Count/Case: {status_change_count/case_count : .2f}" )

    print(f"Adjudicated Cases/FTE Day: {case_count/FTE_DAYS_YEAR : .2f}") 
    print(f"Status Changes/FTE Day: { status_change_count/FTE_DAYS_YEAR: .2f}")
#    print(f"Monthly Team Capacity (Status Changes) { (status_change_count/FTE_DAYS_YEAR) * FTE_DAYS_MONTH: ,.2f}" )


    ftedays_case_count = FTE_DAYS_YEAR/case_count
    ftedays_status_change_count = FTE_DAYS_YEAR/status_change_count
    print(f"FTE Days/Adjudicated Case: {ftedays_case_count : .3f}")
    print(f"FTE Days/Status Change {ftedays_status_change_count: .3f}") 

    return (df, ftedays_case_count, ftedays_status_change_count )

In [ ]:
#exports
def example_exports_func():
    pass



In [ ]:
#exports
def imgs_save(image):
    pass



In [ ]:
#export
def get_statuses(f_path):
    file_path = f_path + 'DFC_STATUSES.csv'
    loadedstatus_df = pd.read_csv(file_path)
    loadedstatus_df = loadedstatus_df.sort_values(by = 'STATUS_CD')
    status_df = loadedstatus_df.drop(columns = ['ACTIVE_YN','UPDATE_RULE','DECISION_CODE','GROUP_ASSIGNMENT'])
    return status_df

In [ ]:

#export
def get_processed_case_data(f_path):
    
    file_path = f_path + 'cases_processed.csv'
    cases_df = pd.read_csv(file_path,parse_dates=['BIRTHDATE','CASE_OPENED_DT','PREV_CASE_END_DT','LAST_STATUS_DATE'], dtype={'DRIVERS_LICENSE_NO': str})
    cases_df = cases_df[(cases_df['Ignore Case'] == 0) ]
    
    cases_df = cases_df[(cases_df['Is Adjudicated'] == 'Adjudicated') & (cases_df['age_bucket'] >= 80)].reset_index()

    cases_df['Origin Report'] = cases_df.apply( lambda x: ORIGIN_MAPPINGS[x['ORIGIN_DSC']] , axis=1)

    cases_df['Origin & Decision'] = cases_df.apply( lambda x: str(x['Origin Report']) + ' & ' + str(x['DECISION_DSC']), axis=1)

    cases_df['First & Second Status'] = cases_df.apply( lambda x: str(x['FIRSTSTATUSCD']) + '_' + str(x['SECOND_CASE_STATUS_CD']), axis=1)

    cases_df['DOCUMENT_COUNT'] = cases_df['DOCUMENT_COUNT'].fillna(0)

    return cases_df

    



In [ ]:

#export
def get_2018processed_case_data(f_path):
    
#     file_path = f_path + 'cases_processed.csv'
#     cases_df = pd.read_csv(file_path,parse_dates=['BIRTHDATE','CASE_OPENED_DT','PREV_CASE_END_DT','LAST_STATUS_DATE'], dtype={'DRIVERS_LICENSE_NO': str})
#     cases_df = cases_df[(cases_df['Ignore Case'] == 0) ]
    cases_df = get_processed_case_data
    cases2018_df = cases_df[cases_df.opened_year == 2018].reset_index()

#     cases2018_df = cases2018_df[(cases2018_df['Is Adjudicated'] == 'Adjudicated') & (cases2018_df['age_bucket'] >= 80)].reset_index()

#     cases2018_df['Origin Report'] = cases2018_df.apply( lambda x: ORIGIN_MAPPINGS[x['ORIGIN_DSC']] , axis=1)

#     cases2018_df['Origin & Decision'] = cases2018_df.apply( lambda x: str(x['Origin Report']) + ' & ' + str(x['DECISION_DSC']), axis=1)

#     cases2018_df['First & Second Status'] = cases2018_df.apply( lambda x: str(x['FIRSTSTATUSCD']) + '_' + str(x['SECOND_CASE_STATUS_CD']), axis=1)

#     cases2018_df['DOCUMENT_COUNT'] = cases2018_df['DOCUMENT_COUNT'].fillna(0)

    status_df = get_statuses(f_path)

    status_df.columns = ['SECOND_STATUS_CD', 'Second Status Desc']
    cases2018_df = pd.merge(cases2018_df, status_df, left_on='SECOND_CASE_STATUS_CD', right_on='SECOND_STATUS_CD' )

    status_df.columns = ['INITIALSTATUSCD', 'Initial Status Desc']
    cases2018_df = pd.merge(cases2018_df, status_df, left_on='FIRSTSTATUSCD', right_on='INITIALSTATUSCD' )

    cases2018_df['First & Second Status Descriptions'] = cases2018_df.apply( lambda x: str(x['Initial Status Desc']) + ' ==> ' + str(x['Second Status Desc']), axis=1)


    cases2018_df = cases2018_df.drop(columns = ['INITIALSTATUSCD', 'SECOND_STATUS_CD'])


    
    return cases2018_df



In [ ]:
#hide
df = get_fte_days_2018()
test_eq( df.shape, (12,2))



In [ ]:
#hide
s = get_statuses(file_path)


In [ ]:
#hide

from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_core.baseline.ipynb.
Converted 02_core.caseload.ipynb.
No export destination, ignored:
#exporti
data_file_path = r'C:\Users\mbeaulieu\rsi_project_book\Data\\'
cases2018_df = get_2018processed_case_data(data_file_path)

Converted 10. Cost Analysis Baseline - 2018 Over 80 Cases by Case Origin.ipynb.
Converted 20. Caseload Modelling Introduction.ipynb.
Converted 21. Caseload Modelling - Monthly Totals Cases Opened.ipynb.
Converted index.ipynb.
